# 🎨 Data Designer 101: Structured Outputs and Jinja Expressions

#### 📚 What you'll learn

In this notebook, we will continue our exploration of Data Designer, demonstrating more advanced data generation using structured outputs and Jinja expressions.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    ExpressionColumnConfig,
    InferenceParameters,
    LLMStructuredColumnConfig,
    ModelConfig,
    PersonSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.


In [2]:
# TODO: fill path to blob storage to be able to access personas
BLOB_STORAGE_PATH = ""

data_designer_client = DataDesigner(blob_storage_path=BLOB_STORAGE_PATH)

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](/models/default-model-settings.md) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

### 🧑‍🎨 Designing our data

- We will again create a product review dataset, but this time we will use structured outputs and Jinja expressions.

- Structured outputs let you specify the exact schema of the data you want to generate.

- Data Designer supports schemas specified using either json schema or Pydantic data models (recommended).

<br>

We'll define our structured outputs using [Pydantic](https://docs.pydantic.dev/latest/) data models

> 💡 **Why Pydantic?**
>
> - Pydantic models provide better IDE support and type validation.
>
> - They are more Pythonic than raw JSON schemas.
>
> - They integrate seamlessly with Data Designer's structured output system.


In [5]:
from decimal import Decimal
from typing import Literal

from pydantic import BaseModel, Field


# We define a Product schema so that the name, description, and price are generated
# in one go, with the types and constraints specified.
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="A description of the product")
    price: Decimal = Field(description="The price of the product", ge=10, le=1000, decimal_places=2)


class ProductReview(BaseModel):
    rating: int = Field(description="The rating of the product", ge=1, le=5)
    customer_mood: Literal["irritated", "mad", "happy", "neutral", "excited"] = Field(
        description="The mood of the customer"
    )
    review: str = Field(description="A review of the product")

Next, let's design our product review dataset using a few more tricks compared to the previous notebook.

> ⚡ **Advanced Feature**:
>
> - Conditional parameters let you modify sampler behavior
>   based on other column values.
>
> - Notice how `review_style` changes when `target_age_range == '18-25'`.


In [6]:
# Since we often only want a few attributes from Person objects, we can
# set drop=True in the column config to drop the column from the final dataset.
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON,
        params=PersonSamplerParams(age_range=[18, 65]),
        drop=False,
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Sampler columns support conditional params, which are used if the condition is met.
# In this example, we set the review style to rambling if the target age range is 18-25.
# Note conditional parameters are only supported for Sampler column types.
config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
        conditional_params={
            "target_age_range == '18-25'": CategorySamplerParams(values=["rambling"]),
        },
    )
)

# Optionally validate that the columns are configured correctly.
config_builder.validate()

[21:51:28] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "customer",
        "product_category",
        "product_subcategory",
        "target_age_range",
        "review_style"
    ]
)

Next, we will use more advanced Jinja expressions to create new columns.

Jinja expressions let you:

- Access nested attributes: `{{ customer.first_name }}`

- Combine values: `{{ customer.first_name }} {{ customer.last_name }}`

- Use conditional logic: `{% if condition %}...{% endif %}`


In [7]:
# We can create new columns using Jinja expressions that reference
# existing columns, including attributes of nested objects.
config_builder.add_column(
    ExpressionColumnConfig(name="customer_name", expr="{{ customer.first_name }} {{ customer.last_name }}")
)

config_builder.add_column(ExpressionColumnConfig(name="customer_age", expr="{{ customer.age }}"))

config_builder.add_column(
    LLMStructuredColumnConfig(
        name="product",
        prompt=(
            "Create a product in the '{{ product_category }}' category, focusing on products  "
            "related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. The product should be priced between $10 and $1000."
        ),
        system_prompt=SYSTEM_PROMPT,
        output_format=Product,
        model_alias=MODEL_ALIAS,
    )
)

# We can even use if/else logic in our Jinja expressions to create more complex prompt patterns.
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="customer_review",
        prompt=(
            "Your task is to write a review for the following product:\n\n"
            "Product Name: {{ product.name }}\n"
            "Product Description: {{ product.description }}\n"
            "Price: {{ product.price }}\n\n"
            "Imagine your name is {{ customer_name }} and you are from {{ customer.city }}, {{ customer.state }}. "
            "Write the review in a style that is '{{ review_style }}'."
            "{% if target_age_range == '18-25' %}"
            "Make sure the review is more informal and conversational."
            "{% else %}"
            "Make sure the review is more formal and structured."
            "{% endif %}"
        ),
        system_prompt=SYSTEM_PROMPT,
        output_format=ProductReview,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.validate()

[21:51:28] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "customer",
        "product_category",
        "product_subcategory",
        "target_age_range",
        "review_style"
    ]
    llm_structured_columns: ['product', 'customer_review']
    expression_columns: ['customer_name', 'customer_age']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [8]:
preview = data_designer_client.preview(config_builder)

[21:51:28] [INFO] 🔁 Preview generation in progress
[21:51:28] [INFO] ✅ Validation passed
[21:51:28] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[21:51:28] [INFO] 🩺 Running health checks for models...
[21:51:28] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[21:51:29] [INFO]   |-- ✅ Passed!
[21:51:29] [INFO] 🎲 Preparing samplers to generate 10 records across 5 columns
[21:51:29] [INFO] 🎲 👨‍💻 Initializing person generation
[21:51:35] [INFO] 🧩 Generating column `customer_name` from expression
[21:51:35] [INFO] 🧩 Generating column `customer_age` from expression
[21:51:35] [INFO] 🗂️ Preparing llm-structured column generation
[21:51:35] [INFO]   |-- column name: 'product'
[21:51:35] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 10

In [9]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer            │ {                                                                                         │
│                     │     'uuid': '1814ab76-7217-4fe7-bbdf-d5c4ac0765c7',                                       │
│                     │     'first_name': 'Joshua',                                                               │
│                     │     'middle_name': 'James',                                                               │
│                     │     'last_name': 'Steiner',                                                               │
│                     │     'sex': 'Male',                                                                        │
│                     │     'age': 64,                                                                            │
│                     │     'marital_status': 'separated',                                                        │
│                     │     'street_name': 'Sabio Way',                                                           │
│                     │     'street_number': 156,                                                                 │
│                     │     'unit': '',                                                                           │
│                     │     'postcode': '65084',                                                                  │
│                     │     'city': 'Versailles',                                                                 │
│                     │     'district': 'Moniteau County',                                                        │
│                     │     'country': 'USA',                                                                     │
│                     │     'bachelors_field': None,                                                              │
│                     │     'education_level': 'some_college',                                                    │
│                     │     'occupation': 'clergy',                                                               │
│                     │     'locale': 'en_US',                                                                    │
│                     │     'birth_date': '1961-03-10',                                                           │
│                     │     'phone_number': '660-638-8615',                                                       │
│                     │     'email_address': 'joshuasteiner96@gmail.com',                                         │
│                     │     'national_id': '490-63-0998',                                                         │
│                     │     'state': 'MO'                                                                         │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_category    │ Home Office                                                                               │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Desks                                                                                     │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 50-65                           

In [10]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,customer,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,{'uuid': '1814ab76-7217-4fe7-bbdf-d5c4ac0765c7...,Home Office,Desks,50-65,brief,Joshua Steiner,64,"{'name': 'Ergonomic Executive Desk', 'descript...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
1,{'uuid': '116c6d83-87f5-4a22-9286-e0a0526ace35...,Clothing,Women's Clothing,50-65,brief,Willis Mashburn,21,"{'name': 'Classic Comfort Dress', 'description...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
2,{'uuid': '61b079ef-beae-4bf3-a946-1f456bf7821c...,Home Office,Storage,18-25,rambling,Christina Quirk,55,"{'name': 'Stackable Under-Bed Storage Bins', '...","{'rating': 4, 'customer_mood': 'happy', 'revie..."
3,{'uuid': '21076af9-117d-402c-ab9e-2947a89b7d1d...,Clothing,Accessories,50-65,rambling,Lisa Williams,24,"{'name': 'Classic Leather Watch Strap', 'descr...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
4,{'uuid': '52e6467e-548e-4e60-8dbb-839176e436c4...,Electronics,Accessories,65+,brief,Stacy Henry,29,"{'name': 'Large Print Smartphone Stand', 'desc...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
5,{'uuid': '2ce0c7b1-2c48-493c-a0ea-a5353e768c5f...,Home Office,Office Supplies,50-65,detailed,Lisa Roberson,41,"{'name': 'Ergonomic Bamboo Desk Organizer', 'd...","{'rating': 5, 'customer_mood': 'happy', 'revie..."
6,{'uuid': '27f11191-ad13-44f2-bcd7-28911ec58cda...,Home Office,Storage,50-65,detailed,Humberto Maldonado,47,{'name': 'Ergonomic Over-the-Door Storage Cabi...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
7,{'uuid': '16388336-4211-47cb-8420-083c4d361a04...,Books,Non-Fiction,25-35,detailed,Alexander Santiago,47,{'name': 'The Power of Habit: Why We Do What W...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
8,{'uuid': '32738f99-a9d7-47fc-93e9-2a534b72f3b9...,Books,Fiction,65+,detailed,Shirley Gooding,61,{'name': 'Whispers of the Willow: A Tale of Lo...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."
9,{'uuid': '36559b4c-95b3-485d-8f03-1f021d2053d6...,Electronics,Accessories,50-65,structured with bullet points,Daniel Lopezvazquez,24,{'name': 'Premium Noise-Cancelling Headphones'...,"{'rating': 5, 'customer_mood': 'happy', 'revie..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [11]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 9                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ customer                         │             dict │                        10 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_category                 │           string │                          4 (40.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                          7 (70.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          4 (40.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (40.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product               │          dict │                10 (100.0%) │     268.0 +/- 0.8 │           69.0 +/- 8.2 │
├───────────────────────┼───────────────┼────────────────────────────┼───────────────────┼────────────────────────┤
│ customer_review       │          dict │                10 (100.0%) │     314.0 +/- 8.5 │         211.5 +/- 82.8 │
└───────────────────────┴───────────────┴───────────────

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [12]:
job_results = data_designer_client.create(config_builder, num_records=20)

[21:51:46] [INFO] 🎨 Creating Data Designer dataset
[21:51:46] [INFO] ✅ Validation passed
[21:51:46] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[21:51:46] [INFO] 🩺 Running health checks for models...
[21:51:46] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[21:51:47] [INFO]   |-- ✅ Passed!
[21:51:47] [INFO] ⏳ Processing batch 1 of 1
[21:51:47] [INFO] 🎲 Preparing samplers to generate 20 records across 5 columns
[21:51:47] [INFO] 🎲 👩‍🎤 Initializing person generation
[21:51:53] [INFO] 🧩 Generating column `customer_name` from expression
[21:51:53] [INFO] 🧩 Generating column `customer_age` from expression
[21:51:53] [INFO] 🗂️ Preparing llm-structured column generation
[21:51:53] [INFO]   |-- column name: 'product'
[21:51:53] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
  

In [13]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,customer,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,"{'age': 24, 'bachelors_field': None, 'birth_da...",Electronics,Cameras,50-65,brief,Luke Williams,24,"{'description': 'A high-quality, easy-to-use d...","{'customer_mood': 'happy', 'rating': 5, 'revie..."
1,"{'age': 58, 'bachelors_field': None, 'birth_da...",Clothing,Men's Clothing,35-50,detailed,Adrian Pappu,58,"{'description': 'A collection of high-quality,...","{'customer_mood': 'happy', 'rating': 5, 'revie..."
2,"{'age': 62, 'bachelors_field': None, 'birth_da...",Books,Classics,25-35,brief,Margie Giles,62,{'description': 'A curated selection of classi...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."
3,"{'age': 48, 'bachelors_field': None, 'birth_da...",Clothing,Accessories,18-25,rambling,Leonard Still,48,{'description': 'A stylish and durable crossbo...,"{'customer_mood': 'happy', 'rating': 4, 'revie..."
4,"{'age': 62, 'bachelors_field': None, 'birth_da...",Home & Kitchen,Cookware,35-50,brief,Beverly Poellnitz,62,{'description': 'A durable and eco-friendly co...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."


In [14]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 9                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ customer                         │             dict │                        20 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_category                 │           string │                          4 (20.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                         13 (65.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (25.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (20.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product               │          dict │                20 (100.0%) │     268.0 +/- 1.0 │           66.5 +/- 9.6 │
├───────────────────────┼───────────────┼────────────────────────────┼───────────────────┼────────────────────────┤
│ customer_review       │          dict │                20 (100.0%) │     311.0 +/- 9.8 │        179.5 +/- 125.0 │
└───────────────────────┴───────────────┴───────────────

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Seeding synthetic data generation with an external dataset](/notebooks/3-seeding-with-a-dataset/)
